In [1]:
import evaluate
import pandas as pd
from src.Case_Builder import (device,
                              bert_version,
                              model_name, 
                              prompt_strategy_used,
                              dataset_name
                              )

In [2]:
dataset_name = 'elife'
results = pd.read_csv(f'results/model_summarie_{bert_version}_{dataset_name}_{prompt_strategy_used}.csv')

In [3]:
rouge = evaluate.load('rouge')
peft_model_results = rouge.compute(
    predictions=results['prediction'],
    references=results['reference'],
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT MODEL:')
print(peft_model_results)

PEFT MODEL:
{'rouge1': np.float64(0.04614943531199689), 'rouge2': np.float64(0.004814983548468742), 'rougeL': np.float64(0.03833422552634375), 'rougeLsum': np.float64(0.03833422552634375)}


In [4]:
results['prediction']

0    The timing of eyelid PCs responses is a functi...
1    mTORC2 is a key protein kinase controlling cel...
2    The 'cortical recycling' circuits are a new ph...
Name: prediction, dtype: object

In [5]:
results['reference']

0    Imagine a gymnastics competition in which part...
1    To grow and multiply , a living cell must take...
2    Our sense of number is thought to have emerged...
Name: reference, dtype: object